# Simulations for the MESS+ estimator

In [1]:
import time

import numpy as np
import pandas as pd
import wandb
import yaml

from pathlib import Path

from classifier.model import MultilabelBERTClassifier
from classifier.file_reader import read_files_from_folder
from classifier.dataset import create_bert_datasets, preprocess_dataframe

from utils.mess_plus import sample_from_bernoulli


INFO 05-06 20:58:02 [__init__.py:243] No platform detected, vLLM is running on UnspecifiedPlatform


In [2]:
BENCHMARK_NAME = "sciq"
# this refers to whether we want to use a pre-trained classifier or learn the classifier online while benchmarking.
APPROACH = "pretrained"  # alt: online
# This only has an effect when APPROACH = pretrained. Make sure to adjust the minibatch size accordingly!
NUM_PRETRAINING_STEPS = 400
SEEDS = [42]
NUM_CLASSIFIER_LABELS = 3

PROJECT_ROOT_PATH = Path("mess_plus_simulator").parent

/home/woi/code/mess-plus/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/woi/code/mess-plus/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:936: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
INFO:utils.misc:All random seeds have been set to 42.


## Load benchmark config

In [3]:

if APPROACH == "pretrained":
	config_path = Path(f"{PROJECT_ROOT_PATH}/config/pretrained/{BENCHMARK_NAME}.yaml")
elif APPROACH == "online":
	config_path = Path(f"{PROJECT_ROOT_PATH}/config/online/{BENCHMARK_NAME}.yaml")
	NUM_PRETRAINING_STEPS = 0
else:
	raise NotImplementedError(f"Approach {APPROACH} not implemented.")

with config_path.open("r") as f:
	CONFIG = yaml.safe_load(f)
	display(CONFIG)


{'run_name': 'baseline',
 'seed': 43,
 'model_zoo': {'meta-llama/Llama-3.2-1B-Instruct': {'category': 'small',
   'gpu_indices': [0],
   'max_seq_len': 2048,
   'gpu_memory_utilization': 0.12,
   'quantization': None},
  'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit': {'category': 'medium',
   'gpu_indices': [0],
   'max_seq_len': 2048,
   'gpu_memory_utilization': 0.15,
   'quantization': 'bitsandbytes'},
  'unsloth/Llama-3.3-70B-Instruct-bnb-4bit': {'category': 'large',
   'gpu_indices': [0],
   'max_seq_len': 2048,
   'gpu_memory_utilization': 0.68,
   'quantization': 'bitsandbytes'}},
 'classifier_model': {'model_id': 'answerdotai/ModernBERT-base',
  'epochs': 3,
  'learning_rate': 0.0362,
  'weight_decay': 0.01,
  'momentum': 0.9,
  'batch_size': 32,
  'max_length': 64,
  'warmup_ratio': 0.1,
  'threshold': 0.5,
  'dropout_rate': 0.1,
  'freeze_bert_layers': True,
  'memory_size': 0,
  'memory_strategy': 'random',
  'reset_optimizer': False,
  'regularization_lambda': 0.0,
  'gpu_i

## Load dataset

In [4]:
input_df = read_files_from_folder(folder_path=f"{PROJECT_ROOT_PATH}/data/inference_outputs/{BENCHMARK_NAME}")
input_df["idx_original"] = input_df.index
input_df = input_df.sample(frac=1).reset_index(drop=True)

display(f"Loaded dataframe with {input_df.shape[0]} rows and {input_df.shape[1]} columns")
display(f"{len(input_df.columns.tolist())} available columns: {input_df.columns.tolist()}")
display(input_df.head())

'Loaded dataframe with 1838 rows and 15 columns'

"15 available columns: ['input_text', 'benchmark_name', 'label_small', 'acc_small', 'energy_consumption_small', 'inference_time_small', 'label_medium', 'acc_medium', 'energy_consumption_medium', 'inference_time_medium', 'label_large', 'acc_large', 'energy_consumption_large', 'inference_time_large', 'idx_original']"

,input_text,benchmark_name,label_small,acc_small,energy_consumption_small,inference_time_small,label_medium,acc_medium,energy_consumption_medium,inference_time_medium,label_large,acc_large,energy_consumption_large,inference_time_large,idx_original
0,Will rising prices correct a recession?,piqa,1.0,1.0,43.305,0.220327,1.0,1.0,210.447,1.034521,1.0,1.0,993.861,2.750166,1594
1,how do I make butterscothcies,piqa,1.0,1.0,22.240,0.138162,1.0,1.0,212.723,1.077468,1.0,1.0,1091.356,2.861364,1376
2,Cut out pages from a book,piqa,0.0,0.0,37.479,0.224905,1.0,1.0,188.012,1.039578,1.0,1.0,1014.041,2.764055,351
3,Remove dents from carpet.,piqa,1.0,1.0,43.358,0.204620,1.0,1.0,192.771,1.029001,1.0,1.0,1022.005,2.739110,1820
4,Trim apple trees.,piqa,0.0,0.0,48.952,0.149915,0.0,0.0,194.003,1.027913,0.0,0.0,1007.760,2.792615,54


## Load, configure, and train classifier

In [5]:
text_col = "input_text"
label_cols = ["label_small", "label_medium", "label_large"]

classifier = MultilabelBERTClassifier(num_labels=NUM_CLASSIFIER_LABELS, **CONFIG["classifier_model"])
training_df = input_df.loc[:NUM_PRETRAINING_STEPS]
training_df = preprocess_dataframe(training_df, label_cols=label_cols)

train_dataset, val_dataset, tokenizer = create_bert_datasets(
	training_df,
	text_col,
	label_cols,
	model_name=CONFIG["classifier_model"]["model_id"],
	max_length=CONFIG["classifier_model"]["max_length"],
	val_ratio=CONFIG["classifier_model"]["validation_dataset_size"],
	random_seed=SEEDS[0],
)

training_stats = classifier.fit(train_dataset, val_dataset, epochs=CONFIG["classifier_model"]["epochs"], early_stopping_patience=2)

display(training_stats)


/home/woi/code/mess-plus/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:classifier.model:Using device: cpu
INFO:classifier.model:Initializing custom BERTClassifier: answerdotai/ModernBERT-base with 3 labels
Epoch 1/3 [Validation]: 100%|██████████| 2/2 [00:00<00:00,  3.71it/s, val_loss=0.5532, avg_val_loss=0.6157]
INFO:classifier.model:Epoch 1/3 - Time: 5.59s
INFO:classifier.model:  Train Loss: 0.6063 - Val Loss: 0.6157
INFO:classifier.model:  Val Metrics - Accuracy: 0.6250, F1: 0.7399, F1(macro): 0.6639
INFO:classifier.model:  Per-label metrics:
INFO:classifier.model:    Label 0: F1=0.2941, Prec=0.6250, Rec=0.1923
INFO:classifier.model:    Label 1: F1=0.8406, Prec=0.7250, Rec=1.0000
INFO:classifier.model:    Label 2: F1=0.8571, Prec=0.7500, Rec=1.0000
INFO:classifier.model:Directory /home/woi/code/mess-plus/classifier/checkpoints/piqa existis. Reusing...
INFO:classifier.model:Best model sav

{'epoch': 3,
 'train/loss': 0.5791598930954933,
 'val/loss': 0.657087653875351,
 'val/accuracy': 0.7,
 'val/precision_micro': 0.7058823529411765,
 'val/recall_micro': 0.9882352941176471,
 'val/f1_micro': 0.8235294117647058,
 'val/f1_macro': 0.822850352262117,
 'time/epoch_seconds': 5.681046724319458,
 'val/0_f1score': 0.7878787878787878,
 'val/0_recall': 0.65,
 'val/0_precision': 1.0,
 'val/1_f1score': 0.8235294117647058,
 'val/1_recall': 0.717948717948718,
 'val/1_precision': 0.9655172413793104,
 'val/2_f1score': 0.8571428571428571,
 'val/2_recall': 0.75,
 'val/2_precision': 1.0}

In [6]:
# Dataset statistics
display(input_df[NUM_PRETRAINING_STEPS:]["label_small"].mean())
display(input_df[NUM_PRETRAINING_STEPS:]["label_medium"].mean())
display(input_df[NUM_PRETRAINING_STEPS:]["label_large"].mean())


0.7475660639777468

0.7906815020862309

0.8261474269819193

In [7]:
algorithm_config = CONFIG["algorithm"]

model_categories = [i for i in CONFIG["model_zoo"].keys()]
sample_cols = input_df.columns.tolist()

ALPHA_VALUES = algorithm_config["alpha_values"]
C_VALUES = [1.0]
V_VALUES = [0.01, 0.001, 0.0001, 0.00001, 0.000001]
R_VALUES = [1]

for alpha in ALPHA_VALUES:
	for c in C_VALUES:
		for v in V_VALUES:
			algorithm_config["V"] = v
			algorithm_config["alpha"] = alpha
			algorithm_config["c"] = c

			ACCURACY_LIST = []
			EXPLORATION_STEP_LIST = []
			ENERGY_CONSUMPTION_LIST = []
			INFERENCE_TIME_LIST = []
			MODEL_CHOSEN_LIST = []

			ENERGY_PER_MODEL = {
				"small": [0.01],
				"medium": [0.1],
				"large": [1.0],
			}

			model_category_list = [i for i in ENERGY_PER_MODEL.keys()]

			Q = 0.0
			ctr = 0

			run = wandb.init(
				project=f"mess-plus_runs_vTEST2",
				name=f"{BENCHMARK_NAME}_V={algorithm_config['V']}_a={algorithm_config['alpha']}_c={algorithm_config['c']}_r={r}",
				config=CONFIG
			)

			if wandb.run is not None:
				run.summary.update({**{f"classifier/{k}": v for k, v in training_stats.items()}})

			monitoring_dict = {}
			for idx, sample in input_df[NUM_PRETRAINING_STEPS:].iterrows():
				p_t, x_t = sample_from_bernoulli(c=algorithm_config["c"], timestamp=idx)
				EXPLORATION_STEP_LIST.append(x_t)

				if x_t == 1:
					result = sample["label_large"]
					ACCURACY_LIST.append(result)
					step_energy = sum([sample[i] for i in sample_cols if "energy" in i])
					step_time = sum([sample[i] for i in sample_cols if "inference" in i])
					ENERGY_CONSUMPTION_LIST.append(step_energy)
					INFERENCE_TIME_LIST.append(step_time)
					for i in ENERGY_PER_MODEL.keys():
						ENERGY_PER_MODEL[i] = sample[f"energy_consumption_{i}"]

					monitoring_dict[f"mess_plus/energy"] = step_energy
					monitoring_dict[f"mess_plus/chosen_model"] = len(model_category_list) - 1

				else:
					preds, probs = classifier.predict(texts=[sample["input_text"]])
					energy = pd.DataFrame(ENERGY_PER_MODEL, index=[0]).to_numpy()

					energy = np.array(energy).reshape(-1, 1)
					probs = probs.reshape(-1, 1)

					cost_fn = algorithm_config["V"] * energy + Q * (alpha - probs)
					cost_fn = cost_fn.reshape(1, -1)
					chosen_model_id = np.argmin(cost_fn)
					# print(f"STEP={ctr} - V={v} - Q={Q} - CHOSEN MODEL: {chosen_model_id} - COST FN: {cost_fn}")
					model_category_chosen = model_category_list[chosen_model_id]

					result = sample[f"label_{model_category_chosen}"]
					step_energy = sample[f"energy_consumption_{model_category_chosen}"]
					step_time = sample[f"inference_time_{model_category_chosen}"]

					INFERENCE_TIME_LIST.append(step_time)
					ENERGY_CONSUMPTION_LIST.append(step_energy)
					MODEL_CHOSEN_LIST.append(chosen_model_id)

					monitoring_dict[f"mess_plus/energy"] = step_energy
					monitoring_dict[f"mess_plus/chosen_model"] = chosen_model_id

					ACCURACY_LIST.append(result)

				Q = max(0.0, Q + algorithm_config["alpha"] - result)

				x = np.array(MODEL_CHOSEN_LIST)
				monitoring_dict.update({
					"mess_plus/p_t": p_t,
					"mess_plus/x_t": x_t,
					"mess_plus/exploration_step_ratio": sum(EXPLORATION_STEP_LIST) / (ctr + 1),
					"mess_plus/q_length": Q,
					"mess_plus/accuracy": sum(ACCURACY_LIST) / (ctr + 1),
					"mess_plus/step_time": step_time,
					"mess_plus/total_runtime": sum(INFERENCE_TIME_LIST),
					"mess_plus/step_energy_consumption": step_energy,
					"models/small_chosen": len(np.where(x == 0)[0]) / (len(x) + 1e-8),
					"models/medium_chosen": len(np.where(x == 1)[0]) / (len(x) + 1e-8),
					"models/large_chosen": len(np.where(x == 2)[0]) / (len(x) + 1e-8),
				})

				print(monitoring_dict)

				ctr += 1
				if wandb.run is not None:
					wandb.log(monitoring_dict, step=ctr)

				if ctr % 3 == 0 and ctr > 0:
					break

			if wandb.run is not None:
				wandb.finish()
				time.sleep(2)


print(f"DONE")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: herbertw (tum-i13). Use `wandb login --relogin` to force relogin


mess_plus/accuracy,▁▄▆▇▆▅▅▅▅▅▆▇▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
mess_plus/chosen_model,▁█▁▁▁▁▁▁██▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁███▁█▁▁▁▁▁
mess_plus/energy,▁▁▁▁▁▁▁█▇█▁▁█▁▁█▁▁█▁▁▁▁▁█▁▁█▁▁▁█▁▁▁▁▁▁▁▁
mess_plus/exploration_step_ratio,▁▁▁▅▅▆▇▇▇▇▇▇▇██▇███▇███▇████████████████
mess_plus/p_t,██▇▇▇▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
mess_plus/q_length,▂▃▃▂▂▅▅▆▆▅▁▂▂▁▁▂▆█▇▄▂▃▃▂▄▅▆▅▅▄▂▃▁▃▄▁▂▂▂▃
mess_plus/step_energy_consumption,▁▁▁▁▁▁█▁▁█▁▁▁▁█▁▇▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁
mess_plus/step_time,▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁██▁█
mess_plus/total_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
mess_plus/x_t,▁▁▁█▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁█▁█▁▁█▁█▁▁█▁█▁▁▁██▁▁
models/large_chosen,▁█▆▇▇███████████████████████████████████


mess_plus/accuracy,▅▁▅▇▇█▆▃▂▃▃▄▃▅▄▂▃▂▂▁▂▂▃▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂
mess_plus/chosen_model,███▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁███▁▁▁▁▁▁▁▁█▁
mess_plus/energy,▁▁█▁▁▁▁▁█▁█▁▇▁▁▁▁▁▁▁▁█▁▁▁▁▁▁███▁▁▁█▁▁▁▁█
mess_plus/exploration_step_ratio,█▃▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁
mess_plus/p_t,██▇▇▇▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
mess_plus/q_length,▂▁▁▁▂▄▂▁▁▁▁█▇█▇▆▂▃▄▃▃▃▆▅▅▆▄▂▁▆▃▄▄▄▁▁▂▁▂▄
mess_plus/step_energy_consumption,███▁▁▁▁▁▁▁▁█▁▁█▁▁█▁▁▁▁█▁▁▁█▁▁▁▁██▁▁▁█▁▁▁
mess_plus/step_time,█▁█▁▁▁▁▁▁▁▁▁▁▁▁██▁▁█▁▁█▁▁▁▁▁▁█▁▁▁▁█▁▁▁██
mess_plus/total_runtime,▁▁▁▂▂▂▂▂▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
mess_plus/x_t,▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁██▁█▁█▁▁█▁█▁▁▁▁█▁▁▁█
models/large_chosen,▆▄▇█▆▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁


mess_plus/accuracy,▄█▆▁▃▁▂▂▂▁▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
mess_plus/chosen_model,█▁█▁██▁▁▁▁▁▁▁▁▁██▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█
mess_plus/energy,▁▁█▁▁█▁▁▁▁█▁█▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁
mess_plus/exploration_step_ratio,█▃▂▂▃▃▃▃▂▂▁▁▁▂▂▂▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁
mess_plus/p_t,████▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
mess_plus/q_length,▁▃▃▃▇▇▆▆▆▃▁▁▃▂▁█▅▇▄▂▃▁▁▁▃▃▂▂▂▃▂▁▃▂▁▁▄▄▇▄
mess_plus/step_energy_consumption,▁█▁▁▁▁▁█▁█▁▁▁▁▁█▁▁█▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁
mess_plus/step_time,▁▁▁███▁█▁▅█▁▁█▁█▁██▁▁█▁█▁▁█▁█▁██▁▁█▁▁▁▁▁
mess_plus/total_runtime,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
mess_plus/x_t,██▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁████▁█▁▁▁▁▁▁█▁█▁
models/large_chosen,▁▂█▃▃▃▃▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂


mess_plus/accuracy,█▅▂▁▂▁▁▁▁▄▄▄▅▅▅▄▃▃▃▃▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
mess_plus/chosen_model,█▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁
mess_plus/energy,█▁▁▁▁▁▁▁█▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁
mess_plus/exploration_step_ratio,█▅▃▃▄▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mess_plus/p_t,███▇▇▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
mess_plus/q_length,▃▅▄▄▃▄▄▅▂▁▁▁▁▄▆▄▅▆▆▅▆▇▅▆▆█▇▆▇▆▄▂▃▃▄▆▅▄▁▁
mess_plus/step_energy_consumption,██▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁
mess_plus/step_time,▁▁▁▁███▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█
mess_plus/total_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇███████
mess_plus/x_t,▁▁▁█▁█▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁█▁▁██▁▁▁▁▁▁▁▁▁▁▁
models/large_chosen,█▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


...

mess_plus/accuracy,▆█▆▇▅▃▃▁▁▁▂▂▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▃▃▃▂▂▂▂▂▂▂▁▁▁
mess_plus/chosen_model,▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁██▁██▁
mess_plus/energy,▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▁▁▁
mess_plus/exploration_step_ratio,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mess_plus/p_t,█▇▇▇▆▆▆▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
mess_plus/q_length,▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▄▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇█
mess_plus/step_energy_consumption,▁▁███▇▁▁▁▁███▁▁█▁▁▁██▁▁▁▁█▇▁▁▁▁▁█▆█▁▁▁█▁
mess_plus/step_time,▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁█▁█▁▁▁██▁▁█▁
mess_plus/total_runtime,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█████
mess_plus/x_t,███▁█▁▁█▁▁█▁▁▁███▁▁▁▁▁▁▁▁█▁▁██▁▁▁█▁█▁▁█▁
models/large_chosen,█▇▅▅▆▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁


mess_plus/accuracy,▃▁▆▄▅▆▄▄▄▄▅▆▇██▇▇▆▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▆▆▆▆
mess_plus/chosen_model,▁▁█▁▁██▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁█▁▁▁▁▁█▁▁▁▁▁▁▁██
mess_plus/energy,▁▁▁██▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁██▁█▁█▁██▁
mess_plus/exploration_step_ratio,█▄▃▃▄▄▄▃▃▃▂▂▂▂▂▂▂▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
mess_plus/p_t,██▇▇▇▇▇▆▆▆▆▆▆▆▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
mess_plus/q_length,▁▁▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇█████
mess_plus/step_energy_consumption,▁▁█▁▇▁▁▁▁▁▁▁▁▁█▁▁▁▁▁██▁▁▁▁▁▁▁▁▆██▁▁▁█▁▁▁
mess_plus/step_time,▁█▁████▁▁▁▁▁▁▁▁▁▁▁█▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁██▁▆
mess_plus/total_runtime,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇█
mess_plus/x_t,▁▁█▁█▁▁▁▁▁█▁▁▁▁█▁▁█▁▁▁██▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁
models/large_chosen,▁▇▆██▇▇▆▆▅▅▅▅▆▅▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄


mess_plus/accuracy,▆█▆▃▄▄▃▂▁▂▃▄▄▆▇▇▅▅▅▅▆▅▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
mess_plus/chosen_model,▁▁█▁▁▁▁███▁█▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁█▁██▁▁▁▁█
mess_plus/energy,██▁██▁▁▁▁█▁██▁▁▁█▁▁▁▁▁▇█▁█▁▁▁▁█▁▁▁▁▁█▁▁▁
mess_plus/exploration_step_ratio,█▄▅▄▄▄▃▃▃▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
mess_plus/p_t,████▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
mess_plus/q_length,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▅▄▄▄▄▄▅▆▆▆▆▆▆▆▇▇▇▇███
mess_plus/step_energy_consumption,▁▇▁▁█▁█▇▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁
mess_plus/step_time,▁█▁█▁█▁███▇▁▁▁▁▁█▁▁█▁▇▁▁▁▁▁▁█▁█▁▁▁▁▁▁█▁▁
mess_plus/total_runtime,▁▁▁▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█████
mess_plus/x_t,▁█████▁▁█▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁▁▁
models/large_chosen,█▄▄▄▃▄▄▃▃▃▃▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁


mess_plus/accuracy,▅█▅▄▂▃▂▃▁▁▁▃▄▆▅▅▆▆▄▃▃▄▃▃▃▃▃▃▃▃▃▃▄▃▃▃▃▃▄▄
mess_plus/chosen_model,█▁▁▁█▁▁██▁█▁▁▁▁▁▁█▁▁▁▁▁█▁█▁▁▁▁▁█▁▁▁▁██▁▁
mess_plus/energy,██▁██▁▁▁▁██▁▁█▁██▁▁▁▁▁▁▁▁█▁█▁▁█▁█▁▁▁██▁▁
mess_plus/exploration_step_ratio,▃▄▇▇█▇▆▆▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mess_plus/p_t,██▇▇▆▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
mess_plus/q_length,▁▁▁▁▁▂▂▂▂▂▃▃▃▂▃▃▃▃▃▄▅▅▅▅▅▅▆▇▇▇▇████▇▇▇▇█
mess_plus/step_energy_consumption,█▁█▁█▁█▁▁▁█▁▁█▁▁▁▁█▁▁▁█▁▁▁▁█▁▁▁▁██▁█▁▁▆▁
mess_plus/step_time,▁▁███▁▁▁▁▁▁█▁▁█▁▁█▁█▁▁█▁▁█▁█▁█▁▁▁▁▁▁▁▁▁▁
mess_plus/total_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
mess_plus/x_t,███▁█▁█▁▁▁▁▁██▁▁▁██▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁
models/large_chosen,▅▆██▇▇▅▅▄▄▄▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


DONE
